In [ ]:
import numpy as np

#class layers
class Layer:
    def forward_propagation(self, input_data):
        raise NotImplementedError

    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError



In [ ]:
# Fully Connected Layer
class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        # Initialisation des poids avec He initialization
        self.weights = np.random.randn(input_size, output_size) * np.sqrt(2. / input_size)
        self.bias = np.zeros((1, output_size))

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * np.sum(output_error, axis=0, keepdims=True)

        return input_error

In [ ]:
# Activation Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return output_error * self.activation_prime(self.input)

In [ ]:
# Sigmoid activation
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    s = sigmoid(x)
    return s * (1 - s)

In [ ]:
# Mean Squared Error
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size

In [ ]:
# Network class
class Network:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)

    def predict(self, input_data):
        output = input_data
        for layer in self.layers:
            output = layer.forward_propagation(output)
        return output

    def train(self, X_train, y_train, loss, loss_prime, epochs=1000, learning_rate=0.1, verbose=True):
        for epoch in range(epochs):
            total_loss = 0
            for x, y in zip(X_train, y_train):
                # Forward pass
                output = self.predict(x.reshape(1, -1))

                # Compute loss
                total_loss += loss(y.reshape(1, -1), output)

                # Backward pass
                error = loss_prime(y.reshape(1, -1), output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            if verbose and (epoch+1) % 1000 == 0:
                print(f"Epoch {epoch+1}, Loss: {total_loss / len(X_train):.6f}")

In [ ]:
# Données XOR
X_train = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

y_train = np.array([
    [0],
    [1],
    [1],
    [0]
])

# Création du réseau
net = Network()
net.add(FCLayer(2, 5))              # Couche d'entrée
net.add(ActivationLayer(sigmoid, sigmoid_prime))
net.add(FCLayer(5, 5))               # Couche cachée
net.add(ActivationLayer(sigmoid, sigmoid_prime))
net.add(FCLayer(5, 1))               # Couche de sortie
net.add(ActivationLayer(sigmoid, sigmoid_prime))

# Entraînement
net.train(X_train, y_train, mse, mse_prime, epochs=10000, learning_rate=0.5)

# Prédictions
print("\nRésultats finaux:")
for x in X_train:
    prediction = net.predict(x.reshape(1, -1))
    binary_pred = 1 if prediction[0][0] > 0.5 else 0
    print(f"Input: {x}, Prédiction: {prediction[0][0]:.4f} (Binaire: {binary_pred})")


Epoch 1000, Loss: 0.000887
Epoch 2000, Loss: 0.000242
Epoch 3000, Loss: 0.000135
Epoch 4000, Loss: 0.000093
Epoch 5000, Loss: 0.000070
Epoch 6000, Loss: 0.000056
Epoch 7000, Loss: 0.000047
Epoch 8000, Loss: 0.000040
Epoch 9000, Loss: 0.000035
Epoch 10000, Loss: 0.000031

Résultats finaux:
Input: [0 0], Prédiction: 0.0048 (Binaire: 0)
Input: [0 1], Prédiction: 0.9948 (Binaire: 1)
Input: [1 0], Prédiction: 0.9944 (Binaire: 1)
Input: [1 1], Prédiction: 0.0065 (Binaire: 0)
